# TESTING AN ONLINE CRYPTO TRADER ADVISOR WITH
- email sending advice
- saving data
- connecting to cex.io and collecting data in real time

In [1]:
from py_files.turtle_buy_strat import *
from py_files.cex_api_wrapper import *

In [2]:
import json
with open('json_files//data.json', 'r') as fp:
        parameters_dict = json.load(fp)
with open('json_files//BTCUSD.json', 'r') as fp:
        dictioBTCUSD = json.load(fp)
with open('json_files//BCHUSD.json', 'r') as fp:
        dictioBCHUSD = json.load(fp)  
with open('json_files//ETHUSD.json', 'r') as fp:
        dictioETHUSD = json.load(fp)
with open('json_files//ETHBTC.json', 'r') as fp:
        dictioETHBTC = json.load(fp)  
with open('json_files//LTCUSD.json', 'r') as fp:
        dictioLTCUSD = json.load(fp)
with open('json_files//XRPUSD.json', 'r') as fp:
        dictioXRPUSD = json.load(fp)  

In [3]:
import time
import pandas as pd
import numpy as np

In [4]:
BTCUSD= pd.read_csv('csv_files//Bitstamp_BTCUSD_1h.csv', skiprows=1, parse_dates=['date'])
BCHUSD= pd.read_csv('csv_files//Bitstamp_BCHUSD_1h.csv', skiprows=1, parse_dates=['date'])
ETHBTC= pd.read_csv('csv_files//Bitstamp_ETHBTC_1h.csv', skiprows=1, parse_dates=['date'])
ETHUSD= pd.read_csv('csv_files//Bitstamp_ETHUSD_1h.csv', skiprows=1, parse_dates=['date'])
LTCUSD= pd.read_csv('csv_files//Bitstamp_LTCUSD_1h.csv', skiprows=1, parse_dates=['date'])
XRPUSD= pd.read_csv('csv_files//Bitstamp_XRPUSD_1h.csv', skiprows=1, parse_dates=['date'])
dfs=[BTCUSD,BCHUSD,ETHBTC,ETHUSD,LTCUSD,XRPUSD] 

In [5]:
TestTurtleBTCUSD=TurtleBuyOnlyOnlineStrategy(dictioBTCUSD)
TestTurtleETHUSD=TurtleBuyOnlyOnlineStrategy(dictioETHUSD)
TestTurtleETHBTC=TurtleBuyOnlyOnlineStrategy(dictioETHBTC)
TestTurtleLTCUSD=TurtleBuyOnlyOnlineStrategy(dictioLTCUSD)
TestTurtleBCHUSD=TurtleBuyOnlyOnlineStrategy(dictioBCHUSD)
TestTurtleXRPUSD=TurtleBuyOnlyOnlineStrategy(dictioXRPUSD)

In [6]:
# some more data to the feeder

In [7]:

a=[ 37838.1,37460.4,37527,37143.5,37488.3,
37526.6,37960.3,38360.9,38475.4,38000,38537,
38383,38353,39050,39638,39583,39290,39500,
39466,39823,39439.8,38800,38965,39202,39401,39189,39224,38706,
  38500,37749,38023.2,38435,38300,38642,38490,38141.5,37666,36991]

b=[494.48,489.92,491.05,477.85,489.32,488,96,
493,498,496,491.19,505,501.5,500,515.65,
512.2,518.83,521.82,525.03,524.57,531.66,
521.15,507.31,510,516.33 ,520,513.01,510.55,
  507.83,499.85,502,507,500,505,500.32,486.22,502,490]

c=[.03,.029839,.029788,.029586,0.029579,
0.020553,0.029597,0.030031,0.030069,
0.030221,0.030306,0.030391,0.030514,
0.030718,0.030558,0.030844,0.031235,
0.031078,0.0307321,0.030863,0.030795,
0.03056,0.030811,0.031419,0.031496,0.031396,0.031422,0.031557,
   0.031392,0.0314,0.32044,0.031737,0.031616,
0.031840,0.031969,0.032269,0.032457,0.032258
   
]

d=[1132.67,1117,1111.07,1099.63,1108.44,1112.7,
1120,1150.47,1159.68,1149.66,1168.16,1167.86,
1168.92,1196.41,1208.01,1214.54,1225.97,
1220.54,1210.52,1230,1206.85,1187.47,1198.6,1229.99,
1230.61,1230.1,1238.86,1217.39,
   1187,1196.31,1235,1216.77,1230.11,1229.89,1229.87,1212.55,1220.39,
   1191.39

]

e=[
146.649,145.987,146.313,144.768,146.384,146.065,
146.885,148.949,148.785,147.729,148.451,147.575,
147.607,152.33,153.072,154.252,154.008,154.828,
154,154,152.477,148.802,150.003,153.198,153.806,150,153.14,150.487,
    147.7,147.915,151,148.707,151.593,149.501,148.801,148.21,149.218,145
]

f=[
0.30305,0.29257,0.28968,0.292,0.29415,0.29598,
0.29601,0.3,0.297,0.29489,0.29688,0.29597,
0.29642,0.29711,0.29977,0.29738,0.3003,
0.2984,0.29698,0.29929,0.295,0.29369,
0.29376,0.297789,0.29766,0.2971,0.29785,0.29502,
    0.29169,0.29101,0.29497,0.29256,0.29084,0.29101,0.29061,0.28884,0.28835,
    0.27785
]

In [8]:
stop=int(TestTurtleBTCUSD.delta1/TestTurtleBTCUSD.delta2)*TestTurtleBTCUSD.window_delta1-len(a)

TestTurtleBTCUSD.initial_feeder(list(BTCUSD.open.values[:stop][::-1])+a)
TestTurtleETHBTC.initial_feeder(list(ETHBTC.open.values[:stop][::-1])+c)
TestTurtleETHUSD.initial_feeder(list(ETHUSD.open.values[:stop][::-1])+d)
TestTurtleLTCUSD.initial_feeder(list(LTCUSD.open.values[:stop][::-1])+e)
TestTurtleBCHUSD.initial_feeder(list(BCHUSD.open.values[:stop][::-1])+b)
TestTurtleXRPUSD.initial_feeder(list(XRPUSD.open.values[:stop][::-1])+f)

In [9]:
username=parameters_dict['CEX_username']
api_key=parameters_dict['CEX_api_key']
api_secret=parameters_dict['CEX_api_secret']
api = Api(username, api_key, api_secret)

online_df_saverBTCUSD=cex_dataframe_online_saver('BTCUSD_price')
online_df_saverBCHUSD=cex_dataframe_online_saver('BCHUSD_price')
online_df_saverETHBTC=cex_dataframe_online_saver('ETHBTC_price')
online_df_saverETHUSD=cex_dataframe_online_saver('ETHUSD_price')
online_df_saverLTCUSD=cex_dataframe_online_saver('LTCUSD_price')
online_df_saverXRPUSD=cex_dataframe_online_saver('XRPUSD_price')


In [ ]:
import time
import datetime as dt 

savers=[ online_df_saverBTCUSD,online_df_saverBCHUSD,online_df_saverETHBTC,
      online_df_saverETHUSD,online_df_saverLTCUSD,online_df_saverXRPUSD]
symbols=['BTC/USD','BCH/USD','ETH/BTC','ETH/USD','LTC/USD','XRP/USD']
turtles=[
    TestTurtleBTCUSD,TestTurtleBCHUSD ,TestTurtleETHBTC ,
    TestTurtleETHUSD ,TestTurtleLTCUSD ,TestTurtleXRPUSD]

while True:
    
    t = dt.datetime.now()   
    if (t.minute==0) : 
        for save,sym,turtle in  zip(savers,symbols,turtles):
            
            tick=api.ticker(sym)
            save.cex_ticker_save(tick)
            feeder={'timestamp':pd.to_datetime( tick['timestamp'][0], unit='s')
                    ,'bid':tick['bid'][0],'ask':tick['ask'][0]}
            turtle.run(feeder)

        time.sleep(600)
        